In [1]:
import pandas as pd
from gnews import GNews
from newspaper import Article, Config
import nltk
import time

# 1. Обязательная настройка библиотек (выполни один раз)
nltk.download('punkt')
nltk.download('punkt_tab')

02/23/2026 12:35:38 AM - Using requests library for http requests (alternative cloudscraper library is recommended for bypassing Cloudflare protection)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\382he\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\382he\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
excluded_domains = [
    'https://www.banki.ru/services/responses',
    'https://smart-lab.ru/blog',
    'https://www.banki.ru/dialog',
    'https://www1.ru'
]

In [3]:
# import pandas as pd
# from gnews import GNews
# from googlenewsdecoder import gnewsdecoder
# import trafilatura
# import requests
# from newspaper import Article, Config
# from sumy.parsers.plaintext import PlaintextParser
# from sumy.nlp.tokenizers import Tokenizer
# from sumy.summarizers.lex_rank import LexRankSummarizer
# import nltk
# import time
# import random

# nltk.download('punkt')

# # --- ИМИТАЦИЯ БРАУЗЕРА ---
# HEADERS = {
#     'User-Agent': 'Yandex (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
#     'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
#     'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
#     'Connection': 'keep-alive'
# }

# PROXIES = {
#     '104.238.30.58:63744',
#     '195.158.8.123:3128'
# }

# def universal_decoder(url):
#     """Декодирует ссылку Google в прямую ссылку на СМИ"""
#     try:
#         decoded_url = gnewsdecoder(url)

#         if decoded_url.get("status"):
#             print("Decoded URL:", decoded_url["decoded_url"])
#             return url
#         else:
#             print("Error:", decoded_url["message"])
#     except Exception as e:
#         print(f"Error occurred: {e}")

# def get_article_text(url, max_retries=5):
#     """Скачивает страницу и извлекает текст"""
#     try:
#         for attempt in range(max_retries):
#             # Используем разные прокси для каждой попытки
#             current_proxy = f"http://{random.choice(PROXIES) if PROXIES else None}"
#             print(f'прокси: {current_proxy}')
#             response = requests.get(
#                 url, 
#                 headers=HEADERS, 
#                 timeout=15, 
#                 allow_redirects=True,
#                 proxies=current_proxy
#             )
#             if response.status_code == 429:
#                 # Extract wait time from Retry-After header if present, otherwise use exponential backoff
#                 retry_after = response.headers.get('Retry-After')
#                 wait_time = int(retry_after) if retry_after else (2 ** attempt) # Exponential backoff
#                 print(f"Rate limited (429). Retrying in {wait_time} seconds...")
#                 time.sleep(wait_time)
#             elif response.status_code == 200:
#                 # Отдаем скачанный HTML в trafilatura
#                 content = trafilatura.extract(response.text, include_comments=False)
#                 return content
#             else:
#                 print(f"⚠️ Ошибка доступа ({response.status_code}) к {url[:30]}...")
#                 return None
#     except Exception as e:
#         print(f"❌ Не удалось загрузить {url[:30]}: {e}")
#         return None

# def get_summary(text, count=2):
#     """Делает выжимку (саммари)"""
#     if not text or len(text) < 200:
#         return "Не удалось извлечь достаточно текста для анализа."
#     try:
#         parser = PlaintextParser.from_string(text, Tokenizer("russian"))
#         summarizer = LexRankSummarizer()
#         summary = summarizer(parser.document, count)
#         return " ".join([str(s) for s in summary])
#     except:
#         return "Ошибка при создании выжимки."
    
# def get_full_data(url):
#     """Функция для извлечения текста и саммари с обходом блокировок"""
#     config = Config()
#     # Имитируем реальный браузер, чтобы сайты не блокировали нас
#     config.browser_user_agent = 'Yandex (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
#     config.request_timeout = 15 # Чтобы скрипт не вис на «мертвых» сайтах
    
#     try:
#         article = Article(url, config=config, language='ru')
#         article.download()
#         article.parse()
#         article.nlp() # Создает то самое summary
        
#         # Если текст слишком короткий, возможно, нас заблокировали
#         if len(article.text) < 5:
#             return "Ошибка: Контент не найден или заблокирован"
            
#         return article.summary
#     except Exception as e:
#         return f"Ошибка парсинга: {e}"
    
# def fetch_history(keyword, start_year, end_year):
#     all_news = []
#     google_news = GNews(language='ru', country='RU', max_results=20) 

#     for year in range(start_year, end_year + 1):
#         print(f"--- Парсим {year} год ---")
#         google_news.start_date = (year, 1, 1)
#         google_news.end_date = (year, 12, 31)
        
#         results = google_news.get_news(keyword)
        
#         for item in results:
#             print(f"Обработка: {item['title'][:50]}...")
            
#             real_url = universal_decoder(item['url'])

#             summary = get_full_data(real_url)
            
#             all_news.append({
#                 'date': item['published date'],
#                 'title': item['title'],
#                 'url': real_url,
#                 'source': item['publisher']['title'],
#                 'summary': summary
#             })
#             time.sleep(1) # Пауза, чтобы не забанили
            
#             # # Получаем первые 2 страницы (около 20-30 статей)
#             # for page in range(1, 3):
#             #     gn.get_page(page)
#             #     for item in gn.results():
#             #         real_url = universal_decoder(item['link'])
#             #         print(f"🔗 Парсим: {item['title'][:50]}...")
                    
#             #         full_text = get_article_text(real_url)
#             #         if full_text == None:
#             #             continue
                    
#             #         summary = get_summary(full_text)
                    
#             #         all_news.append({
#             #             'date': item['date'],
#             #             'title': item['title'],
#             #             'url': real_url,
#             #             'summary': summary
#             #         })
#             #     gn.clear()
            
#             # Сохраняем промежуточный результат
#             pd.DataFrame(all_news).to_csv('news_checkpoint.csv', index=False, encoding='utf-8-sig')
#             time.sleep(random.uniform(2, 4)) # Пауза для обхода капчи

#     return pd.DataFrame(all_news)

# # --- ЗАПУСК ---
# KEYWORD = "сбербанк"
# df = fetch_history(KEYWORD, 2025, 2025) # Для теста возьмем один год

# # Итоговое сохранение
# df.to_csv(f'news_{KEYWORD}.csv', index=False, encoding='utf-8-sig')
# print(f"\n✅ Готово! Собрано статей: {len(df)}")

In [4]:
import pandas as pd
from gnews import GNews
import time
import random
import trafilatura
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from sumy.parsers.plaintext import PlaintextParser
from selenium.webdriver.common.by import By
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from selenium.common.exceptions import NoSuchElementException
from nltk.tokenize import sent_tokenize
import nltk
from datetime import datetime, timedelta
import os

nltk.download('punkt')
nltk.download('punkt_tab')

# --- НАСТРОЙКА БРАУЗЕРА ---
def init_driver():
    chrome_options = Options()
    # chrome_options.add_argument("--headless") # Раскомментируй, чтобы скрыть окно браузера
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # ГЛАВНОЕ: Игнорируем ошибки сертификатов (Минцифры и прочие)
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument("--allow-insecure-localhost")
    chrome_options.add_argument("--ignore-ssl-errors=yes")
    # Маскируемся под обычного пользователя, чтобы не спалили автоматизацию
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def get_summary(text, max_sentences=4):
    if not text or len(text) < 100: return "Текст слишком короткий"
    
    # 1. ПРЕДВАРИТЕЛЬНАЯ ЧИСТКА
    # Заменяем переносы строк на пробелы, чтобы склеенные строки разошлись
    clean_text = text.replace("\n", " ").replace("\r", " ")
    
    # Убираем лишние пробелы (два и более подряд)
    clean_text = " ".join(clean_text.split())

    try:
        # 2. Sumy выбирает "лучший кусок" текста
        parser = PlaintextParser.from_string(clean_text, Tokenizer("russian"))
        summarizer = LexRankSummarizer()
        
        # Просим Sumy дать 1 предложение. 
        # Но если токенизатор ошибся, Sumy вернет огромный кусок.
        sumy_result = summarizer(parser.document, max_sentences)
        raw_summary = " ".join([str(s) for s in sumy_result])
        
        # 3. ФИНАЛЬНАЯ НАРЕЗКА (SAFETY NET)
        # Мы берем то, что вернула Sumy, и еще раз режем на предложения через NLTK
        real_sentences = sent_tokenize(raw_summary, language="russian")
        
        if real_sentences:
            # Возвращаем СТРОГО первое предложение
            return ' '.join(real_sentences[:max_sentences])
        else:
            return raw_summary
            
    except Exception as e:
        print(f"Ошибка в NLP: {e}")
        return "Ошибка обработки"

def fetch_with_selenium(keyword, start_date, end_date):
    # Запускаем браузер один раз
    driver = init_driver()
    all_news = []
    
    google_news = GNews(language='ru', country='RU', max_results=100, exclude_websites=excluded_domains)

    try:
        print(f"--- парсим {end_date-start_date} начиная с {start_date} ---")
        google_news.start_date = start_date
        google_news.end_date = end_date
        
        results = google_news.get_news(keyword)
        
        print(f"Найдено ссылок: {len(results)}")
        
        for i, item in enumerate(results):
            url = item['url']
            title = item['title']
            
            print(f"[{i+1}] Открываю: {title[:40]}...")
            
            try:
                # 1. Selenium открывает ссылку (Google сам сделает редирект в браузере)
                driver.get(url)
                
                # 2. Ждем, пока прогрузятся скрипты и редиректы
                time.sleep(2) 
                
                # 3. Забираем готовый HTML после всех загрузок
                html = driver.page_source
                
                # 4. Проверка на заглушку сертификатов
                if "Национального УЦ Минцифры" in html:
                    print("⚠️ Попали на страницу сертификатов. Пробуем искать кнопку перехода...")
                    # Тут можно добавить клик по кнопке "Все равно перейти", если она есть
                    # Но обычно ignore-certificate-errors в опциях выше решает это
                
                # 5. Вытаскиваем текст
                text = trafilatura.extract(html, include_comments=False)
                date = driver.find_element(By.CLASS_NAME, "time")
                
                if text:
                    summary = get_summary(text)
                    # Получаем реальный URL из адресной строки браузера
                    real_url = driver.current_url 
                    
                    all_news.append({
                        'date': item['published date'],
                        'scraped_date': date,
                        'title': title,
                        'url': real_url,
                        'summary': summary
                    })
                    print(f"✅ Успех! Длина текста: {len(text)}")
                else:
                    print("❌ Текст не найден (Trafilatura вернула пустоту)")
                    
            except NoSuchElementException:
                print(f'не нашли время на сайте {url}')

            except Exception as e:
                print(f"❌ Ошибка драйвера: {e}")

    finally:
        # Обязательно закрываем браузер в конце
        driver.quit()
        
    return pd.DataFrame(all_news)

# --- ЗАПУСК ---
KEYWORD = "сбербанк"
start_date = datetime(2026, 1, 1)
end_date = datetime(2026, 2, 20)   
WINDOW = 2 # сколько дней прибавлять
print("Запускаем браузер...")

dfs = []
current_date = start_date
while current_date <= end_date:
    current_window_end = current_date + timedelta(days=WINDOW)
    print(f"\n--- Обработка недели: {current_date.strftime('%Y-%m-%d')} - {current_window_end.strftime('%Y-%m-%d')} ---")
    
    try:
        week_df = fetch_with_selenium(KEYWORD, current_date, current_window_end)
        
        if not week_df.empty:
            # Если файл уже существует, дописываем без заголовка
            if os.path.isfile(f'{KEYWORD}_{WINDOW}day(s)_news.csv'):
                week_df.to_csv(f'{KEYWORD}_{WINDOW}day(s)_news.csv', mode='a', header=False, index=False, encoding='utf-8-sig')
            else:
                week_df.to_csv(f'{KEYWORD}_{WINDOW}day(s)_news.csv', index=False, encoding='utf-8-sig')
            
            print(f"Сохранено {len(week_df)} статей за неделю")
        else:
            print("Нет данных за эту неделю")

    except Exception as e:
        print(f"Критическая ошибка на неделе: {e}")
        # Можно записать лог ошибки, но цикл продолжается
    
    current_date = current_window_end

print("\nГотово! Проверяй файл.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\382he\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\382he\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
02/23/2026 12:35:39 AM - ====== WebDriver manager ======


Запускаем браузер...

--- Обработка недели: 2026-01-01 - 2026-01-03 ---


02/23/2026 12:35:40 AM - Get LATEST chromedriver version for google-chrome
02/23/2026 12:35:40 AM - Get LATEST chromedriver version for google-chrome
02/23/2026 12:35:40 AM - Driver [C:\Users\382he\.wdm\drivers\chromedriver\win64\145.0.7632.77\chromedriver-win32/chromedriver.exe] found in cache


--- парсим 2 days, 0:00:00 начиная с 2026-01-01 00:00:00 ---
Найдено ссылок: 39
[1] Открываю: Сбербанк ввел новую плату за обслуживани...
не нашли время на сайте https://news.google.com/rss/articles/CBMilgFBVV95cUxOVmxsQ2dFOFdkU3hoSjNGY0ZxaDhLdUtvcW5QNmtndkRnZlZjS1ZyaklPY2lua1RBdFdTR1dzTDdNRWEwQ2djUHo5am5IUXpGdU9HNVZXTGROTDh4ZXNGVk81RDIzNDM0SGpuOHdUZWdMTnFwTjFqZldXbGZSQjFoOVZ0Q01vNW44VHhxU2J1MEIxYjdGc3c?oc=5&hl=en-US&gl=US&ceid=US:en
[2] Открываю: Звонят с номера 900: Сбербанк объяснил, ...
не нашли время на сайте https://news.google.com/rss/articles/CBMitAFBVV95cUxNOG1vMHRSX2hXb1JlUEVWVjJfRXpOcTRkQW9hTGxTckdDYTE1b252Mmkxb2FhZGMwcGRKeVBhV3ZSYjFSanVzY3M4SVNVQUhvT1BubU85Rno1R2swOGZmREh4V0Z5VjZablpidXFUdjA3RmZXWGtKczNDcjZ1RWVJbWFzV2lOWUlpYlZQRW5oVjBuVFFEVnVGeTN6RXF1MUlzcHdNUHZrbXlqYV83MHppUnBzdFDSAboBQVVfeXFMUHRpazVLSWh2UHp6ME4wSGdpLUlTN1dlVmFndV9kX1h1bHc1YXVnOXVBQ29UcTgzOExYVmdYMzZ6R3U2ZUNjT0hqOTBWT1R0RTVRM0FTbl9BNEJtb0RaTXpvR0FURVcwQUEzLUJtSlotTnZtNGVSazU1aXJHMXVTbnhiVzJDNmxSOEw4OUoyak9

02/23/2026 12:40:17 AM - ====== WebDriver manager ======


Сохранено 1 статей за неделю

--- Обработка недели: 2026-01-03 - 2026-01-05 ---


02/23/2026 12:40:18 AM - Get LATEST chromedriver version for google-chrome
02/23/2026 12:40:19 AM - Get LATEST chromedriver version for google-chrome
02/23/2026 12:40:19 AM - Driver [C:\Users\382he\.wdm\drivers\chromedriver\win64\145.0.7632.77\chromedriver-win32/chromedriver.exe] found in cache


--- парсим 2 days, 0:00:00 начиная с 2026-01-03 00:00:00 ---


KeyboardInterrupt: 

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
tokenizer = Tokenizer("ru")

# https://ru.m.wikipedia.org/wiki/Тунберг,_Грета
sentences = tokenizer.to_sentences("Гре́та Тинтин Элеонора Э́рнман Ту́нберг (швед. Greta Tintin Eleonora Ernman Thunberg; род. 3 января 2003[1][2], Стокгольм[1]) — шведская школьница, экологическая активистка. В 15 лет начала протестовать возле шведского парламента с плакатом «Школьная забастовка за климат», призывая к незамедлительным действиям по борьбе с изменением климата в соответствии с Парижским соглашением. Её действия нашли отклик по всему миру, породив массовые мероприятия, известные как «школьные забастовки за климат» или «пятницы ради будущего»")

for sentence in sentences:
    print(sentence)
    # print(tokenizer.to_words(sentence))

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(task='text-classification', model='ProsusAI/finbert')

In [ ]:
def calculate_sentiment_index(neutral, positive, negative):
    # Приводим к шкале 0...1, где 0.5 - чистый нейтрал
    sentiment_index = (positive * 1.0) + (neutral * 0.5) + (negative * 0.0)
    return round(sentiment_index, 4)

known_acronyms = ['руб.', 'млн.', 'трлн.', 'гос.', 'тыс.']

def calculate_market_index(summary):
    result_all = pipe(summary, top_k=None)
    try:
        idx = calculate_sentiment_index(result_all[0]['score'], result_all[1]['score'], result_all[2]['score'])
    except Exception as e:
        if hasattr(e, 'Token indices sequence length is longer than the specified maximum sequence length for this model'):
            return None # здесь должна быть функция обработки текста, превышающего заданную длину
    return idx

In [ ]:
df['market_index'] = df['summary'].apply(calculate_market_index)
print(df['market_index'])